# LIME: Applied for RoBERTa

In [ ]:
import argparse
import torch
import transformers
from pathlib import Path

from models.roberta import RobertaUrl
from models.decision_tree import DecisionTreeUrl
from trainers.roberta_trainer import RobertaTrainer
from trainers.decision_tree_trainer import DecisionTreeTrainer
from data.feature_extractor import FeatureExtractor
import utils.torch_utils as ptu
from data.data_utils import load_url_dataset
from trainers.trainer_metrics import TrainerMetrics
from utils.utils import add_default_arguments
from interpretability.url_explainer import UrlExplainer

%load_ext autoreload
%autoreload 2

In [15]:
def load_model(model_name: str, experiment_name: str, device):
    experiment_dir = Path("models/trained") / experiment_name
    # Return the model, tokenizer, and trainer class for the given model name
    if model_name == "roberta":
        model = RobertaUrl()
        model.load_state_dict(torch.load(experiment_dir / "roberta_url.pth"))
        return model.to(device), transformers.RobertaTokenizer.from_pretrained("roberta-base"), RobertaTrainer
    elif model_name == "decision_tree":
        model = DecisionTreeUrl()
        model.load_state_dict(experiment_dir / "decision_tree_url.pkl")
        return model, None, DecisionTreeTrainer
    else:
        raise ValueError(f"Invalid model name: {model_name}")
    
def load_config_data(experiment_name: str):
    # Load the config from the experiment directory
    experiment_dir = Path("models/trained") / experiment_name
    metrics = TrainerMetrics.from_file(experiment_dir / "metrics.json", config_only=True)
    
    model_name = metrics.config["model"]
    batch_size = metrics.config["batch_size"]

    return model_name, batch_size


In [18]:
splits_directory = Path("data/dataset_1/splits")
experiment_name = "dt"
model_name = "decision_tree"
batch_size = 32

(_, _, testloader), classes = load_url_dataset(splits_directory, batch_size, num_workers=4, test=True)

max_seq_length = 512 # Load that from the dataset config
device = ptu.get_device()

# Load the model
model, tokenizer, trainer_cls = load_model(model_name, experiment_name, device)
loss_fn = torch.nn.BCELoss()

trainer = trainer_cls(experiment_name,
                    model,
                    tokenizer,
                    loss_fn,
                    None,
                    None,
                    None,
                    None,
                    testloader,
                    classes,
                    device=device,
                    max_seq_length=512)

/home/local/USHERBROOKE/levj1404/Documents/ift714/UDS-IFT714-LanguesNaturelles/trainers/base_trainer.py:13: UserWarning: Experiment dt already exists. Files might be overwritten.
  warnings.warn(f"Experiment {self.experiment_name} already exists. Files might be overwritten.")


In [19]:
trainer_explainer = trainer
explainer = UrlExplainer(save_dir="explanations", class_names=["benign", "malicious"])
urls_to_explain = [
    "paypal.com.cgi.bin.webscr.cmd.flow.session.lohzumu98pjkwkwudgtj3ie6btlub.online775885d80a13c0db1f8e263663d3faee8d43b1bb6ca3ufquez.login.efzg5epaloginuk.uzmandoktorum.com/"
]

explanations = explainer.explain_list(urls_to_explain, trainer_explainer.predict)

/home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 99 features, but DecisionTreeClassifier is expecting 112 features as input.